!pip install seaborn emcee --user

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sb
import emcee
import pandas as pd

In [ ]:
sb.set_style('white')

# Crecimiento Exponencial

$\frac{dG}{dt} = rG$,

donde G representa el numero de personas a un tiempo data y r es la tasa de crecimiento. La solucion general a esta ecuación diferencial es:

$G = Ae^{rt}$,

donde A es una normalización.

# Crecimiento Logístico

$\frac{dG}{dt} = rG\left(1-\frac{G}{K}\right)$,

donde el parametro nuevo, K, es capacidad máxima de población que un sistema ecológico puede soportar.La solución a esta ecuación es:

$G = \frac{K}{1+Ae^{-rt}}$

In [ ]:
def exp_growth(pars,time):
    r,logA = pars
    A = 10**logA
    return A*np.exp(r*time)

In [ ]:
def logistic(pars,time):
    r,logK,logA = pars
    A = 10**logA
    K = 10**logK
    val = K / (1. + A*np.exp(-r*time))
    return val

In [ ]:
class Growth_Models(object):
    def __init__(self,time,model):
        self.model = model
        self.time = time

    def __call__(self,pars):
        if self.model == 'exp':
            return exp_growth(pars,self.time)
        if self.model == 'logistic':
            return logistic(pars,self.time)

In [ ]:
def lnhood(pars,data,err,time,model):
    init_fun = Growth_Models(time,model)
    model_val = init_fun(pars)

    p = ((data - model_val)**2 / err**2) + np.log(2. * np.pi * err**2)
    return -0.5 * np.sum(p)

In [ ]:
def priors(pars,plist,model):
    if model == 'exp':
        r,A = pars
        if plist[0]<r<plist[1] and plist[2]<A<plist[3]:
            return 0.0
        return -np.inf
    if model == 'logistic':
        r,K,A = pars
        if plist[0]<r<plist[1] and plist[2]<K<plist[3] and\
            plist[4]<A<plist[5]:
            return 0.0
        return -np.inf

In [ ]:
def lnpost(pars,data,err,time,plist,model):
    p = priors(pars,plist,model)
    if not np.isfinite(p):
        return -np.inf
    return p + lnhood(pars,data,err,time,model)

# Graficamos los datos primero

In [ ]:
data = np.loadtxt('Datos_Poblacionales_Gdl.dat').T

In [ ]:
plt.errorbar(data[0],data[1],yerr=data[2],fmt='o',color='orangered');
plt.xlabel('Year');
plt.ylabel('Number of people');

In [ ]:
time = np.arange(0,80,np.diff(data[0])[0])

In [ ]:
print len(time)

# Maximizamos el Likelihood para inicializar nuestros caminadores en una zona de alta probabilidad

empezamos con el modelo exponencial

In [ ]:
fun = lambda *args: -lnhood(*args)

In [ ]:
from scipy import optimize as op

In [ ]:
result = op.minimize(fun,[0.5,6.,3.0],args=(data[1],data[2],time,'logistic'),method='TNC')

In [ ]:
print result.x

In [ ]:
plt.plot(data[0],logistic(result.x,time),'g',label='Inference');
plt.errorbar(data[0],data[1],yerr=data[2],fmt='o',color='orangered',label='data');
plt.xlabel('Year');
plt.ylabel('Number of people');
plt.legend(loc='best');

# Ahora sigue el MCMC

In [ ]:
prior_list = [0.,1.,0,8.0,0.,8.]

In [ ]:
nwalkers = 60
ndim = (len(prior_list)/2) 

In [ ]:
z = np.zeros((ndim,nwalkers))
h = 1e-2

pos_i=[]

for i in range(ndim):
    z[i,:] = result.x[i] + h*np.random.randn(nwalkers)

for i in range(nwalkers):
    pos_i.append(np.array([z[0,i],z[1,i],z[2,i]]))

In [ ]:
b_steps, steps = 150,500

In [ ]:
sampler = emcee.EnsembleSampler(nwalkers,ndim,lnpost, 
                                args=(data[1],data[2],time,prior_list,'logistic'),
                                threads = 2)

In [ ]:
pos,prob,state=sampler.run_mcmc(pos_i, b_steps)

In [ ]:
print sampler.acceptance_fraction.mean()

In [ ]:
sampler.reset()

In [ ]:
_,_,_=sampler.run_mcmc(pos, steps, rstate0=state)

In [ ]:
print sampler.acceptance_fraction.mean()

# Graficamos las cadenas para checar convergencia 

In [ ]:
plt.figure()
plt.plot(sampler.flatchain[:,0]);
plt.figure()
plt.plot(sampler.flatchain[:,1]);
plt.figure()
plt.plot(sampler.flatchain[:,2])

In [ ]:
chains_df = pd.DataFrame(sampler.flatchain,
                         columns=['r','log(K)','log(A)'])

In [ ]:
c = sb.PairGrid(chains_df)
c.map_lower(sb.kdeplot, cmap="Blues_d")
c.map_diag(plt.hist,bins=50,histtype='step')
for i,j in zip(*np.triu_indices_from(c.axes, 1)):
    c.axes[i,j].set_visible(False)

In [ ]:
models_exp=[]
for i in range(len(sampler.flatchain[:,0])):
    models_exp.append(logistic([sampler.flatchain[i,0],
                                sampler.flatchain[i,1],
                               sampler.flatchain[i,2]],time))

In [ ]:
print np.shape(models_exp)

In [ ]:
mean_exp = np.mean(models_exp,axis=0)
std_exp = np.std(models_exp,axis=0)

In [ ]:
plt.errorbar(data[0],data[1],yerr=data[2],fmt='o',color='orangered',label = 'data');
plt.plot(data[0],mean_exp,'g',label='inference');
plt.fill_between(data[0],mean_exp-std_exp,mean_exp+std_exp,alpha=0.2,color='g');
plt.xlabel('Year',fontsize=14);
plt.ylabel('Number of people',fontsize=14);
plt.legend(loc='best');